##### Imports

In [1]:
import sys
from pathlib import Path
import warnings

import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

import random
from collections import Counter
from scr.utils import read_words
import gc


import sys
# Custom library paths
sys.path.extend(['../', './scr'])

from scr.utils import set_seed
from scr.utils import read_words

# set_seed(42)

import torch
import torch.nn as nn

torch.set_float32_matmul_precision('medium')

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##### Data Reading and Feature Engineering

In [2]:
"""
This code cell performs the following operations for a Hangman game dataset preparation:

1. Set a constant `NUM_STRATIFIED_SAMPLES` for the number of samples to be used.

2. Check and define the base directory path for storing the dataset. The base directory is named based on the number of stratified samples and is located within a 'dataset' folder in the current working directory.

3. Create the base directory if it doesn't exist, along with a subdirectory named 'pkl' for storing pickle files.

4. Define paths for training, testing, and validation word lists within the base directory. These are expected to be text files containing words for each respective dataset split.

5. Check if these files exist in the specified paths. If they do, read the words from these files into respective variables (`train_words`, `test_words`, `val_words`). If any of the files are not found, print a message indicating the file is missing.

Note: The functions `read_words` and `write_words` used in the code are assumed to be defined in the 'scr.utils' module for reading and writing words from/to text files.
"""

import os
from pathlib import Path
from scr.utils import *

NUM_STRATIFIED_SAMPLES = 1000

# # Check current working directory
# current_dir = os.getcwd()
# print("Current Working Directory:", current_dir)

# Define the base directory where you want to save the datase/t
base_dataset_dir = Path(f'./dataset/{NUM_STRATIFIED_SAMPLES}')
# Define the base directory on the specified drive
# base_dataset_dir = Path(f"/media/sayem/510B93E12554BBD1/dataset/{NUM_STRATIFIED_SAMPLES}")

# Ensure the base directory exists
base_dataset_dir.mkdir(parents=True, exist_ok=True)

pkls_dir = base_dataset_dir / 'pkl'
pkls_dir.mkdir(parents=True, exist_ok=True)

# Adjust these paths if necessary
train_words_path = base_dataset_dir / 'train_words.txt'
test_words_path = base_dataset_dir / 'test_words.txt'
val_words_path = base_dataset_dir / 'val_words.txt'

# Check if files exist before reading
if train_words_path.exists():
    train_words = read_words(train_words_path)
else:
    print(f"File not found: {train_words_path}")

if test_words_path.exists():
    test_words = read_words(test_words_path)
else:
    print(f"File not found: {test_words_path}")

if val_words_path.exists():
    val_words = read_words(val_words_path)
else:
    print(f"File not found: {val_words_path}")


File not found: dataset/1000/train_words.txt
File not found: dataset/1000/test_words.txt
File not found: dataset/1000/val_words.txt


##### Testing Single Word inferece Pipeline

In [3]:
# # For inference
from scr.feature_engineering import *

# from scr.feature_engineering import build_feature_set, \
#     process_single_word_inference, calculate_char_frequencies, \
#         calculate_word_frequencies
from scr.utils import *

word_list = read_words('data/words_250000_train.txt')
word_frequencies = calculate_word_frequencies(word_list)
char_frequency = calculate_char_frequencies(word_list)
max_word_length = max(len(word) for word in word_list)

# # Example usage
# # Example usage
# game_state = "_pp_e"
# max_seq_length = 6  # Set this to the maximum number of turns in your game

# # Now include max_seq_length in the function call
# # for testing the function
# sequence_features, sequence_missed_chars = \
#     process_game_sequence([game_state], char_frequency, max_word_length, max_seq_length)

In [4]:
# batch_size = 1
# # Reshaping to separate features
# reshaped_features = sequence_features.view(batch_size, max_seq_length, max_word_length, -1)
# # Extracting the character indices
# char_indices = reshaped_features[:, :, :, 0]

In [5]:
# # Example of processing a batch of games
# batch_of_games = [['_ppl_']]
# char_frequency = calculate_char_frequencies(word_list)
# max_word_length = max_word_length  # Assuming this is the maximum word length
# max_seq_length = max_seq_length  # Maximum number of states (guesses) per game

# # Example of processing a batch of games
# batch_features, batch_missed_characters = process_batch_of_games(batch_of_games, \
#     char_frequency, max_word_length, max_seq_length)
# print(batch_features.shape)  # Expected shape: [2, 6, max_word_length * num_features]
# print(batch_missed_characters.shape)  # Expected shape: [2, 6, len(char_to_idx)]

In [6]:
# batch_size = 1
# # Reshaping to separate features
# reshaped_features = batch_features.view(batch_size, max_seq_length, max_word_length, -1)
# # Extracting the character indices
# char_indices = reshaped_features[:, :, :, 0]

In [7]:
# # Example of processing a batch of games
# batch_of_games = [['_ppl_', 'appl_', 'apple'], ['_a__l_', 'ba__l_', 'ball_'], ['_a__l_', 'ba__l_', 'ball_']]
# # batch_of_games = [['_ppl_']]
# batch_size = len(batch_of_games)
# char_frequency = calculate_char_frequencies(word_list)
# max_word_length = max_word_length  # Assuming this is the maximum word length
# max_seq_length = max_seq_length  # Maximum number of states (guesses) per game

# # Example of processing a batch of games
# batch_features, batch_missed_characters \
#     = process_batch_of_games(batch_of_games, \
#         char_frequency, max_word_length, max_seq_length)
# print(batch_features.shape)  # Expected shape: [2, 6, max_word_length * num_features]
# print(batch_missed_characters.shape)  # Expected shape: [2, 6, len(char_to_idx)]

In [8]:
# # Reshaping to separate features
# reshaped_features = batch_features.view(batch_size, \
#     max_seq_length, max_word_length, -1)
# # Extracting the character indices
# char_indices = reshaped_features[:, :, :, 0]

##### Model Building

In [9]:
from scr.simple_model import SimpleLSTM
from scr.base_model import BaseModel

# Instantiate and test the model
config = {
    'embedding_dim': 200,
    'hidden_dim': 256,
    'num_layers': 2,
    'vocab_size': 27,
    'max_word_length': max_word_length,
    'input_feature_size': 5,
    'use_embedding': True,
    'miss_linear_dim': 50
}

model = SimpleLSTM(config)
# x_lens = torch.tensor([max_seq_length] * batch_size, dtype=torch.long)
# output = model(batch_features, x_lens, batch_missed_characters)
# with torch.no_grad():
#     output = model(batch_features, x_lens, batch_missed_characters)
#     print("Output shape:", output.shape)

# Assuming 'model' is your trained model instance
model.save_model(file_path='models/model.pth')

In [10]:
from pathlib import Path
import torch

# Assuming the saved model file is 'models/model.pth'
model_file_path = 'models/model.pth'

# Specify the device to load the model onto
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model
loaded_model = BaseModel.load_model(SimpleLSTM, model_file_path)
# Now `loaded_model` is an instance of `SimpleLSTM` with the state and config loaded

##### Playing Games Untrain NN

In [11]:
%%capture
"""
Hangman Game Simulation with Neural Network Integration
--------------------------------------------------------

Overview:
---------
This code is designed to integrate a neural network (NN) model into a Hangman game simulation. 
It tests the model's ability to predict characters in the context of the game, using an untrained model 
as a baseline. The code covers several aspects, including character prediction, game state updates, 
and overall game logic.

Key Components:
---------------
1. `guess_character` Function:
   - Core function that predicts the next character in the Hangman game.
   - Takes parameters like the current game state, character frequency, NN model, and device configuration.
   - Processes the current masked word and predicts the next character using the model's output.
   - Applies softmax to get a probability distribution and excludes already guessed characters.
   - Implements a fallback strategy to choose the most common unguessed character if needed.

2. `guess` Function:
   - Wrapper function that calls `guess_character`.
   - Cleans and preprocesses the word for guessing.
   - Manages the list of already guessed letters.

3. `play_game_with_a_word` Function: mimic api:
   - Simulates an entire Hangman game with a given word.
   - Manages the game state, including guessed letters, remaining attempts, and masked word.
   - Calls `guess` function to make guesses and updates the game state accordingly.
   - Determines the game outcome (win/lose) based on the guesses and the word to be guessed.

4. Testing with Sample Words:
   - The code tests the game logic with a set of sample words.
   - Each word is used to play a complete game, showcasing the model's predictions and the game's progression.
   - The results (win/lose, final word state, and number of attempts used) are printed for each test word.

Purpose of Testing:
-------------------
- To assess the NN model's character prediction capabilities in a game scenario.
- To verify the game logic, including the updating of the game state and the effectiveness of the fallback strategy.
- To identify areas for potential improvement or adjustment in both the model and the game logic.

Importance:
-----------
- This testing phase is crucial for understanding how the model behaves with different inputs and varying game states.
- It provides insights into the model's performance before and after training, highlighting its learning progress.
- The tests help ensure that the integration of the model with the Hangman game logic is functioning as expected.

Next Steps:
-----------
- Evaluate the model's performance with these initial tests and identify areas for improvement.
- Train the model with appropriate datasets to enhance its prediction accuracy.
- Continuously test the model with new game states to ensure robust performance.
- Refine the game logic and fallback strategies based on test outcomes.

Conclusion:
-----------
This setup forms a foundational part of developing an AI-powered Hangman game. 
It bridges the gap between NN models and game mechanics, 
paving the way for creating an intelligent game-playing agent.

"""

In [12]:
from scr.game import guess_character, guess

# Define the game state
game_state = '_a_'  # The current state of the word being guessed

# Define dummy guessed letters (if any have been guessed)
guessed_letters = []  # No letters guessed yet

# Call the guess function
# # Testing the guess fucntion with NN integration
model = loaded_model
# model.to(device)  # Make sure model is on the correct device
# # tensor = tensor.to(device)
char = guess(model, game_state, char_frequency, \
        max_word_length, device, guessed_letters)

# Print the guessed character
print("Guessed Character:", char)

Guessed Character: x


In [13]:
from scr.game import play_game_with_a_word
game_state = 'apple'
won, final_word, attempts = play_game_with_a_word(model, game_state, char_frequency, \
    max_word_length, device, max_attempts=6, normalize=True)

print(won, final_word, attempts)

False _____ 6


In [14]:
words = ['apple', 'kale', 'moon', 'missisippi']
set_seed(412)

# Model interacting with my/dummy api
# For validation
for word in words:
    print(word)
    won, final_word, attempts = play_game_with_a_word(
        model, word, char_frequency, \
        max_word_length, device, max_attempts=6, normalize=True)

    print(won, final_word, attempts)
    print()

apple
False _____ 6

kale
False ____ 6

moon
False moo_ 6

missisippi
False mi__i_ippi 6



##### Building Dataset From States

In [15]:
# ## Reading all pkls and inpect

# pkl_list = []

# for batch_dir in sorted(pkls_dir.iterdir(), key=lambda x: int(x.name) \
#     if x.name.isdigit() else float('inf')):
#     if batch_dir.is_dir():
#         pkl_files = list(batch_dir.glob("*.pkl"))

#         for pkl_file in pkl_files:
#             with open(pkl_file, 'rb') as file:
#                 game_data = pickle.load(file)

#                 # print(game_data)

#                 # Splitting the file name
#                 parts = pkl_file.stem.split('_from_')
#                 word = parts[0]  # Extracting the word
#                 # Further splitting to extract initial state, difficulty, and outcome
#                 remaining = parts[1].split('_')
#                 initial_state = '_'.join(remaining[:-2])  # Joining to form the initial state
#                 # print(initial_state)
#                 difficulty, outcome = remaining[-2], remaining[-1]

#                 for state_data in game_data:
#                     game_won, guesses = state_data
#                     scenario = {
#                         'word': word,
#                         'initial_state': initial_state,
#                         'difficulty': difficulty,
#                         'outcome': outcome,
#                         'data': (game_won, guesses)
#                     }
#                     pkl_list.append((pkl_file, scenario))

# index_to_access = 1000
# if index_to_access < len(pkl_list):
#     file_path, scenario = pkl_list[index_to_access]
#     print(f"Contents of {file_path}:")
#     print()
#     print_scenarios([scenario])
# else:
#     print(f"No pickle file at index {index_to_access}")


# print(list(pkl_list[0]))

In [16]:
pkls_dir

PosixPath('dataset/1000/pkl')

In [17]:
from scr.dataset import ProcessedHangmanDataset
from sklearn.model_selection import train_test_split

# # Load the dataset
processed_dataset = ProcessedHangmanDataset(pkls_dir, \
    char_frequency, max_word_length)

# Convert PyTorch dataset to a list for train_test_split
dataset_list = [processed_dataset[i] for i in range(len(processed_dataset))]

# # Perform an 80%-20% train-test split
# train_data, test_data = train_test_split(dataset_list, test_size=0.20, random_state=42)

In [18]:
dataset_list

[]

In [19]:
print(len(processed_dataset))

0


In [20]:
processed_dataset[100]

IndexError: list index out of range

In [ ]:
processed_dataset[1000]

(['_en_ood', '_en_ood', '_en_ood', '_en_ood', '_en_ood', '_en_ood', 'ken_ood'],
 ['a', 'u', 'm', 'h', 's', 'k', 'b'],
 {'word': 'kenwood',
  'initial_state': '_en_oo_',
  'difficulty': 'hard',
  'outcome': 'win'})

In [ ]:
%%capture
"""
[
    ['_ _ _ _', '_ a _ _', 'a p _ _'],  # Game 1
    ['_ _ _ _', '_ _ _ l', '_ a _ l'],  # Game 2
    # ... more games ...
]

"""

In [ ]:
from torch.nn.utils.rnn import pad_sequence
import torch

def custom_collate_fn(batch):
    batch_features, batch_missed_chars, batch_labels, batch_lengths = [], [], [], []
    max_seq_length = 0  # Variable to track the maximum sequence length in the batch

    # First, process each game to find the maximum sequence length
    for item in batch:
        game_states, labels, _ = item
        if not game_states:
            continue
        max_seq_length = max(max_seq_length, len(game_states))

    # Now, process each game again to pad sequences and collect batch data
    for item in batch:
        game_states, labels, _ = item
        if not game_states:
            continue

        game_features, game_missed_chars = process_game_sequence(
            game_states, char_frequency, max_word_length, len(game_states))

        # Record the original length of each game state sequence
        original_length = len(game_states)
        batch_lengths.append(original_length)

        # Pad each game's features and missed characters to the maximum sequence length
        if original_length < max_seq_length:
            padding_length = max_seq_length - original_length

            # Create padding tensor for game_features
            padding_tensor_features = torch.zeros(padding_length, game_features.shape[1])
            game_features_padded = torch.cat([game_features, padding_tensor_features], dim=0)

            # Create a separate padding tensor for game_missed_chars
            padding_tensor_missed_chars = \
                torch.zeros(padding_length, game_missed_chars.shape[1])
            game_missed_chars_padded = \
                torch.cat([game_missed_chars, padding_tensor_missed_chars], dim=0)
        else:
            game_features_padded = game_features
            game_missed_chars_padded = game_missed_chars

        batch_features.append(game_features_padded)
        batch_missed_chars.append(game_missed_chars_padded)
        batch_labels.extend([char_to_idx[label] for label in labels])

    # Before stacking, check if the lists are empty
    if not batch_features or not batch_missed_chars:
        # Handle empty batch here, maybe skip or return None
        print("Encountered an empty batch")
        return None, None, None, None

    # Stack all games to form the batch
    batch_features_stacked = torch.stack(batch_features)
    batch_missed_chars_stacked = torch.stack(batch_missed_chars)
    labels_tensor = torch.tensor(batch_labels, dtype=torch.long)
    lengths_tensor = torch.tensor(batch_lengths, dtype=torch.long)

    return batch_features_stacked, lengths_tensor, batch_missed_chars_stacked, labels_tensor


In [ ]:
from scr.custom_sampler import PerformanceBasedSampler

import random
from collections import defaultdict  # Add this line
from torch.utils.data import Sampler

# The rest of your PerformanceBasedSampler class and test function remains the same


def test_performance_based_sampler(dataset, performance_metrics, \
    target_win_rate=0.6, max_weight=120):
    # Create the sampler
    sampler = PerformanceBasedSampler(dataset, performance_metrics, \
        target_win_rate, max_weight)

    # Generate indices using the sampler
    indices = sampler.generate_indices()

    # Analyze the distribution of the sampled data
    distribution_analysis = analyze_sampler_distribution(indices, dataset)

    # # Print the results for review
    # print("Sampler Distribution Analysis:")
    # print(distribution_analysis)

    return analyze_sampler_distribution(indices, dataset)

    # Additional checks can be performed here based on specific requirements
    # For example, checking if certain types of words are over/underrepresented

def analyze_sampler_distribution(indices, dataset):
    # print(f"Number of indices: {len(indices)}")  # Debug print
    # if indices:
    #     print(f"Sample index: {indices[0]}")  # Debug print

    distribution = {
        'word_length': defaultdict(int),
        'difficulty': defaultdict(int),
        'outcome': defaultdict(int)
    }

    for idx in indices:
        _, _, additional_info = dataset[idx]
        word_length = len(additional_info['word'])
        difficulty = additional_info['difficulty']
        outcome = additional_info['outcome']

        distribution['word_length'][word_length] += 1
        distribution['difficulty'][difficulty] += 1
        distribution['outcome'][outcome] += 1

    return distribution

# # Call the test function again
# test_performance_based_sampler(processed_dataset, \
#     performance_metrics, target_win_rate=0.6, max_weight=120)

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path

def plot_sampler_distribution(distribution_analysis, save_path=None, epoch=None):
    # Plotting settings
    plt.figure(figsize=(15, 5))

    # Plot for word length distribution
    plt.subplot(1, 3, 1)
    plt.bar(distribution_analysis['word_length'].keys(), \
        distribution_analysis['word_length'].values(), color='blue')
    plt.title("Word Length Distribution")
    plt.xlabel("Word Length")
    plt.ylabel("Frequency")

    # Plot for difficulty distribution
    plt.subplot(1, 3, 2)
    plt.bar(distribution_analysis['difficulty'].keys(), \
        distribution_analysis['difficulty'].values(), color='green')
    plt.title("Difficulty Distribution")
    plt.xlabel("Difficulty")
    plt.ylabel("Frequency")

    # Plot for outcome distribution
    plt.subplot(1, 3, 3)
    plt.bar(distribution_analysis['outcome'].keys(), \
        distribution_analysis['outcome'].values(), color='red')
    plt.title("Outcome Distribution")
    plt.xlabel("Outcome")
    plt.ylabel("Frequency")

    plt.tight_layout()

    # Determine the path for saving the plot using pathlib
    if save_path is None:
        directory = Path('plots/sampler_dist')
        directory.mkdir(parents=True, exist_ok=True)
        file_name = f"epoch_{epoch}.png" if epoch is not None else "untrain_model_stats.png"
        save_path = directory / file_name

    plt.savefig(save_path)
    plt.close()

# Example usage
# plot_sampler_distribution(your_distribution_data, save_path=None, epoch=1)  # Save for a specific epoch
# plot_sampler_distribution(your_distribution_data, save_path=None)  # Save for untrained model


# Example usage
# plot_sampler_distribution(your_distribution_data, save_path=None, epoch=1)  # Save for a specific epoch
# plot_sampler_distribution(your_distribution_data, save_path=None)  # Save for untrained model


# # # Use this function to plot the distribution analysis
# distribution_analysis = test_performance_based_sampler(processed_dataset, \
#     performance_metrics, target_win_rate=0.6, max_weight=120)
# plot_sampler_distribution(distribution_analysis)


In [ ]:
import torch.nn.functional as F

def pad_and_reshape_labels(labels, model_output_shape):
    batch_size, sequence_length, num_classes = model_output_shape

    # Calculate the total number of elements needed
    total_elements = batch_size * sequence_length

    # Pad the labels to the correct total length
    padded_labels = F.pad(input=labels, pad=(0, total_elements - labels.numel()), value=0)

    # Reshape the labels to match the batch and sequence length
    reshaped_labels = padded_labels.view(batch_size, sequence_length)

    # Convert to one-hot encoding
    one_hot_labels = F.one_hot(reshaped_labels, num_classes=num_classes).float()

    return one_hot_labels

In [ ]:
from torch.utils.data import DataLoader
from scr.custom_sampler import PerformanceBasedSampler

# Initialize your custom sampler with the dataset and performance metrics
sampler = PerformanceBasedSampler(processed_dataset, performance_metrics)

# Set batch size
batch_size = 64

# Initialize DataLoader with the custom sampler, instead of shuffle
# data_loader = DataLoader(processed_dataset, \
#     batch_size=batch_size, sampler=sampler, collate_fn=custom_collate_fn)

data_loader = DataLoader(processed_dataset, \
    batch_size=batch_size, collate_fn=custom_collate_fn)

i = 0
# Iterate over the DataLoader
for i, batch in enumerate(data_loader):
    if batch[0] is None:
        print("Encountered an empty batch")
        continue  # Skip empty batches

    game_states_batch, lengths_batch, missed_chars_batch, labels_batch = batch
    
    print("Game States Batch Shape:", game_states_batch.shape)
    print("Lengths Batch Shape:", lengths_batch.shape)
    print("Missed Chars Batch Shape:", missed_chars_batch.shape)
    print("Labels Batch Shape:", labels_batch.shape)

    # # Assuming 'model' is your trained model
    out = model(game_states_batch, lengths_batch, missed_chars_batch)
    # Usage example
    model_output_shape = out.shape  # Assuming 'out' is the model output with shape [32, 9, 27]
    reshaped_labels = pad_and_reshape_labels(labels_batch, model_output_shape)
    print("Model Output Shape:", model_output_shape)
    print("reshape_labels shape: ", reshaped_labels.shape)

    loss, miss_penalty = model.calculate_loss(out, reshaped_labels, lengths_batch, missed_chars_batch, \
        27)

    # print(loss)

    # # # print(i)
    # break


In [ ]:
STOP

##### Training Pipeline

In [ ]:
from torch.nn.utils.rnn import pad_sequence
import torch
from torch.utils.data import DataLoader


def train_on_data_loader(model, data_loader, device, optimizer):
    model.train()  # Set the model to training mode
    model.to(device)
    total_loss = 0
    total_miss_penalty = 0
    total_batches = 0

    for batch in data_loader:
        if batch[0] is None:
            print("Encountered an empty batch")
            continue  # Skip empty batches

        game_states_batch, lengths_batch, missed_chars_batch, labels_batch = batch
        game_states_batch, lengths_batch, missed_chars_batch \
            = game_states_batch.to(device), \
            lengths_batch, missed_chars_batch.to(device)

        # Assuming 'model' is your trained model
        outputs = model(game_states_batch, lengths_batch, missed_chars_batch)

        # Reshape labels to match model output
        reshaped_labels = pad_and_reshape_labels(labels_batch, outputs.shape)
        reshaped_labels = reshaped_labels.to(device)

        # Compute loss and miss penalty
        loss, miss_penalty = model.calculate_loss(outputs, reshaped_labels, \
            lengths_batch, missed_chars_batch, 27)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_miss_penalty += miss_penalty.item()
        total_batches += 1

    average_loss = total_loss / total_batches if total_batches > 0 else 0
    average_miss_penalty = total_miss_penalty / total_batches if total_batches > 0 else 0
    return average_loss, average_miss_penalty


In [ ]:
from scr.feature_engineering import idx_to_char, char_to_idx

def validate_hangman(model, data_loader, device):
    model.eval()
    total_loss = 0
    total_miss_penalty = 0
    correct_char_predictions = 0
    correct_word_predictions = 0
    total_char_predictions = 0
    total_words = 0

    with torch.no_grad():
        for batch in data_loader:
            batch_features_tensor, batch_missed_chars_tensor, \
                batch_labels_tensor, batch_full_words = batch

            # game_states_batch, guessed_chars_batch, labels_batch = \
            #     game_states_batch.to(device), guessed_chars_batch.to(device), labels_batch.to(device)

            batch_size = 1
            max_seq_length = 1

            sequence_lengths = torch.tensor([max_seq_length]
                                            * batch_size, dtype=torch.long).cpu()

            # Model's inference for character prediction
            output = model(batch_features_tensor, sequence_lengths, batch_missed_chars_tensor)

            # Calculate loss and miss penalty - assuming model has a method for this
            loss, miss_penalty = model.calculate_loss(output, labels_batch, guessed_chars_batch)
            total_loss += loss.item()
            total_miss_penalty += miss_penalty.item()

            for idx, (sequence_length, full_word) in enumerate(zip(guessed_chars_batch, full_words)):
                last_char_position = sequence_length.item() - 1
                probabilities = torch.softmax(output[idx, last_char_position, :], dim=-1)
                
                # Exclude already guessed characters
                guessed_indices = [char_to_idx[char] for char in guessed_chars_batch[idx] if char in char_to_idx]
                probabilities[torch.tensor(guessed_indices, dtype=torch.long, device=device)] = 0

                # Predict the best character
                best_char_index = torch.argmax(probabilities).item()
                guessed_char = idx_to_char[best_char_index]

                # Update character-level accuracy
                correct_char_predictions += int(guessed_char == full_word[last_char_position])
                total_char_predictions += 1

                # Update word-level accuracy
                predicted_word = ''.join([idx_to_char[char_idx] for char_idx in output[idx].argmax(dim=-1)])
                correct_word_predictions += int(predicted_word == full_word)

            total_words += len(full_words)

    avg_loss = total_loss / len(data_loader)
    avg_miss_penalty = total_miss_penalty / len(data_loader)
    char_accuracy = correct_char_predictions / total_char_predictions
    word_accuracy = correct_word_predictions / total_words

    return avg_loss, avg_miss_penalty, char_accuracy, word_accuracy

In [ ]:
from collections import defaultdict

def validate_model(model, val_words, char_frequency, \
    max_word_length, device, max_attempts=6, normalize=True, \
        number_of_games=1000):
    total_wins, total_attempts, total_games = 0, 0, 0
    word_stats = {}
    word_length_stats = defaultdict(lambda: {"wins": 0, \
        "losses": 0, "total_attempts": 0, "games": 0})

    for word in val_words:
        if total_games >= number_of_games:
            break

        word_length = len(word)
        masked_word = "_" * word_length
        won, final_word, attempts = play_game_with_a_word(model, word, \
            char_frequency, max_word_length, device, max_attempts, normalize)

        # Update word-specific stats
        word_stats[word] = {
            "won": won,
            "final_word": final_word,
            "attempts_used": attempts
        }

        # Update word length-specific stats
        word_length_stats[word_length]["games"] += 1
        word_length_stats[word_length]["total_attempts"] += attempts
        if won:
            word_length_stats[word_length]["wins"] += 1
        else:
            word_length_stats[word_length]["losses"] += 1

        # Update overall stats
        total_wins += int(won)
        total_attempts += attempts
        total_games += 1

    win_rate = total_wins / total_games if total_games > 0 else 0
    avg_attempts = total_attempts / total_games if total_games > 0 else 0

    # Calculating detailed statistics for each word length
    for length, stats in word_length_stats.items():
        stats["win_rate"] = stats["wins"] / stats["games"]
        stats["avg_attempts"] = stats["total_attempts"] / stats["games"]

    return {
        "win_rate": win_rate,
        "average_attempts": avg_attempts,
        "total_games": total_games,
        "total_wins": total_wins,
        "total_losses": total_games - total_wins,
        "word_stats": word_stats,
        "word_length_stats": dict(word_length_stats)  # Convert defaultdict to dict for readability
    }

from collections import defaultdict

def calculate_character_metrics(final_word, original_word):
    true_positives, false_positives, false_negatives = defaultdict(int), \
        defaultdict(int), defaultdict(int)

    for char in set(original_word):
        if char in final_word:  # If the char is in the guessed word
            if char in original_word:  # If the char is also in the original word
                true_positives[char] += 1
            else:
                false_positives[char] += 1
        else:
            if char in original_word:  # If the char is in the original word but not in the guess
                false_negatives[char] += 1

    return true_positives, false_positives, false_negatives

##### Performence testing of Untrain Model

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path
from scr.plot_utils import plot_word_stats

validation_on_dataset = validate_model(model, val_words, \
    char_frequency, max_word_length, device, max_attempts=6, \
        normalize=True, number_of_games=1000)

# Example usage
# epoch_number = 1
save_path = 'plots/untrain_model_stats.png'  # Example custom path
plot_word_stats(validation_on_dataset['word_length_stats'], save_path)

##### Train

In [ ]:
def update_performance_metrics(metrics, validation_results):
    """
    Update the initial performance metrics based on the results of the validation.

    Args:
    - metrics (dict): The current performance metrics.
    - validation_results (dict): The results from the validation phase.

    Returns:
    - None: The function updates the metrics in place.
    """
    for word_length, stats in validation_results.get('word_length_stats', {}).items():
        win_rate = stats.get("win_rate", 0)
        avg_attempts = stats.get("avg_attempts", 0)

        # Ensure the word length entry exists
        if word_length not in metrics:
            metrics[word_length] = {"easy": {}, "medium": {}, "hard": {}}

        for difficulty in ["easy", "medium", "hard"]:
            # Update or initialize metrics for each difficulty level
            metrics[word_length][difficulty]['win'] = {"win_rate": win_rate, "max_attempts": avg_attempts}
            metrics[word_length][difficulty]['lose'] = {"win_rate": 1 - win_rate, "max_attempts": avg_attempts + 1}


In [ ]:
from scr.simple_model import SimpleLSTM
from scr.base_model import BaseModel

# Instantiate and test the model
config = {
    'embedding_dim': 200,
    'hidden_dim': 256,
    'num_layers': 2,
    'vocab_size': 27,
    'max_word_length': max_word_length,
    'input_feature_size': 5,
    'use_embedding': True,
    'miss_linear_dim': 50,
    'lr': 0.0001
}

model = SimpleLSTM(config)
# x_lens = torch.tensor([max_seq_length] * batch_size, dtype=torch.long)
# output = model(batch_features, x_lens, batch_missed_characters)
# with torch.no_grad():
#     output = model(batch_features, x_lens, batch_missed_characters)
#     print("Output shape:", output.shape)

# Assuming 'model' is your trained model instance
model.save_model(file_path='models/model.pth')

In [ ]:
import random

def generate_initial_performance_metrics(word_lengths, \
    difficulties, win_rate_range, max_attempts_range):
    metrics = {}
    for length in word_lengths:
        metrics[length] = {}
        for difficulty in difficulties:
            win_rate = random.uniform(*win_rate_range)
            max_attempts = random.randint(*max_attempts_range)
            metrics[length][difficulty] = {
                "win": {"win_rate": win_rate, "max_attempts": max_attempts},
                "lose": {"win_rate": 1 - win_rate, "max_attempts": max_attempts + 1}  
                # Slightly higher max_attempts for 'lose' scenario
            }
    return metrics

# Example usage
word_lengths = [4, 5, 6]  # Example word lengths
difficulties = ["easy", "medium", "hard"]  # Difficulty categories
win_rate_range = (0.4, 0.7)  # Range for random win rates
max_attempts_range = (5, 10)  # Range for random max attempts

initial_performance_metrics = \
    generate_initial_performance_metrics(word_lengths, \
        difficulties, win_rate_range, max_attempts_range)

print(initial_performance_metrics)

{4: {'easy': {'win': {'win_rate': 0.636346420154477, 'max_attempts': 8}, 'lose': {'win_rate': 0.36365357984552305, 'max_attempts': 9}}, 'medium': {'win': {'win_rate': 0.6653949260720473, 'max_attempts': 10}, 'lose': {'win_rate': 0.3346050739279527, 'max_attempts': 11}}, 'hard': {'win': {'win_rate': 0.6623314774505196, 'max_attempts': 7}, 'lose': {'win_rate': 0.33766852254948043, 'max_attempts': 8}}}, 5: {'easy': {'win': {'win_rate': 0.5286305149608064, 'max_attempts': 10}, 'lose': {'win_rate': 0.4713694850391936, 'max_attempts': 11}}, 'medium': {'win': {'win_rate': 0.6069551925106758, 'max_attempts': 5}, 'lose': {'win_rate': 0.39304480748932424, 'max_attempts': 6}}, 'hard': {'win': {'win_rate': 0.6668464773909362, 'max_attempts': 5}, 'lose': {'win_rate': 0.33315352260906383, 'max_attempts': 6}}}, 6: {'easy': {'win': {'win_rate': 0.4793910104401178, 'max_attempts': 5}, 'lose': {'win_rate': 0.5206089895598822, 'max_attempts': 6}}, 'medium': {'win': {'win_rate': 0.5735922430947795, 'max_a

In [ ]:
distribution_analysis = test_performance_based_sampler(processed_dataset, \
    performance_metrics=initial_performance_metrics, target_win_rate=0.6, max_weight=120)
plot_sampler_distribution(distribution_analysis)

In [ ]:
import torch
from torch.utils.data import DataLoader
from pathlib import Path
import gc
from tqdm import tqdm
from scr.custom_sampler import PerformanceBasedSampler, update_sampler
from torch.optim.lr_scheduler import ReduceLROnPlateau


# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Assuming the existence of necessary functions, model setup, and collate_fn

num_epochs = 20
batch_size = 256
# initial_performance_metrics = {}

shuffle = True

optimizer = model.optimizer

# Initialize the scheduler for the optimizer
scheduler = ReduceLROnPlateau(model.optimizer, mode='max', factor=0.5, \
    patience=2, verbose=True)

# plot_word_stats(word_length_stats)

all_epochs_performance_metrics = {}

for epoch in tqdm(range(num_epochs), desc='Epochs'):
    
    distribution_analysis = test_performance_based_sampler(processed_dataset, \
                    performance_metrics=initial_performance_metrics, \
                        target_win_rate=0.6, max_weight=120)
    
    plot_sampler_distribution(distribution_analysis, epoch=epoch)

    # print(initial_performance_metrics)
    
    # Update the sampler for the current epoch
    sampler = PerformanceBasedSampler(processed_dataset, initial_performance_metrics)
    data_loader = DataLoader(processed_dataset, \
        batch_size=batch_size, sampler=sampler, collate_fn=custom_collate_fn)

    # Train on this epoch's data
    avg_loss, avg_miss_penalty = train_on_data_loader(model, \
        data_loader, device, optimizer)

    # Validation step
    model.eval()
    with torch.no_grad():
        validation_results = validate_model(model, val_words, char_frequency, \
            max_word_length, device, max_attempts=6, normalize=True, number_of_games=1000)
    
    # Update performance metrics and scheduler
    
    update_performance_metrics(initial_performance_metrics, validation_results)
    scheduler.step(validation_results['win_rate'])

    print(initial_performance_metrics)

    # Print epoch summary
    print(f"Epoch {epoch+1}: Training - Avg Loss: {avg_loss}, Avg Miss Penalty: {avg_miss_penalty}")
    print(f"Epoch {epoch+1}: Validation - Win Rate: {validation_results['win_rate']}")

    gc.collect()

print('Training completed')


Epochs:   5%|▌         | 1/20 [00:13<04:24, 13.92s/it]

{4: {'easy': {'win': {'win_rate': 0.13333333333333333, 'max_attempts': 5.833333333333333}, 'lose': {'win_rate': 0.8666666666666667, 'max_attempts': 6.833333333333333}}, 'medium': {'win': {'win_rate': 0.13333333333333333, 'max_attempts': 5.833333333333333}, 'lose': {'win_rate': 0.8666666666666667, 'max_attempts': 6.833333333333333}}, 'hard': {'win': {'win_rate': 0.13333333333333333, 'max_attempts': 5.833333333333333}, 'lose': {'win_rate': 0.8666666666666667, 'max_attempts': 6.833333333333333}}}, 5: {'easy': {'win': {'win_rate': 0.03636363636363636, 'max_attempts': 5.890909090909091}, 'lose': {'win_rate': 0.9636363636363636, 'max_attempts': 6.890909090909091}}, 'medium': {'win': {'win_rate': 0.03636363636363636, 'max_attempts': 5.890909090909091}, 'lose': {'win_rate': 0.9636363636363636, 'max_attempts': 6.890909090909091}}, 'hard': {'win': {'win_rate': 0.03636363636363636, 'max_attempts': 5.890909090909091}, 'lose': {'win_rate': 0.9636363636363636, 'max_attempts': 6.890909090909091}}}, 6

Epochs:  10%|█         | 2/20 [00:27<04:10, 13.92s/it]

{4: {'easy': {'win': {'win_rate': 0.13333333333333333, 'max_attempts': 5.833333333333333}, 'lose': {'win_rate': 0.8666666666666667, 'max_attempts': 6.833333333333333}}, 'medium': {'win': {'win_rate': 0.13333333333333333, 'max_attempts': 5.833333333333333}, 'lose': {'win_rate': 0.8666666666666667, 'max_attempts': 6.833333333333333}}, 'hard': {'win': {'win_rate': 0.13333333333333333, 'max_attempts': 5.833333333333333}, 'lose': {'win_rate': 0.8666666666666667, 'max_attempts': 6.833333333333333}}}, 5: {'easy': {'win': {'win_rate': 0.03636363636363636, 'max_attempts': 5.890909090909091}, 'lose': {'win_rate': 0.9636363636363636, 'max_attempts': 6.890909090909091}}, 'medium': {'win': {'win_rate': 0.03636363636363636, 'max_attempts': 5.890909090909091}, 'lose': {'win_rate': 0.9636363636363636, 'max_attempts': 6.890909090909091}}, 'hard': {'win': {'win_rate': 0.03636363636363636, 'max_attempts': 5.890909090909091}, 'lose': {'win_rate': 0.9636363636363636, 'max_attempts': 6.890909090909091}}}, 6

Epochs:  15%|█▌        | 3/20 [00:41<03:57, 13.95s/it]

...

{4: {'easy': {'win': {'win_rate': 0.13333333333333333, 'max_attempts': 5.833333333333333}, 'lose': {'win_rate': 0.8666666666666667, 'max_attempts': 6.833333333333333}}, 'medium': {'win': {'win_rate': 0.13333333333333333, 'max_attempts': 5.833333333333333}, 'lose': {'win_rate': 0.8666666666666667, 'max_attempts': 6.833333333333333}}, 'hard': {'win': {'win_rate': 0.13333333333333333, 'max_attempts': 5.833333333333333}, 'lose': {'win_rate': 0.8666666666666667, 'max_attempts': 6.833333333333333}}}, 5: {'easy': {'win': {'win_rate': 0.03636363636363636, 'max_attempts': 5.890909090909091}, 'lose': {'win_rate': 0.9636363636363636, 'max_attempts': 6.890909090909091}}, 'medium': {'win': {'win_rate': 0.03636363636363636, 'max_attempts': 5.890909090909091}, 'lose': {'win_rate': 0.9636363636363636, 'max_attempts': 6.890909090909091}}, 'hard': {'win': {'win_rate': 0.03636363636363636, 'max_attempts': 5.890909090909091}, 'lose': {'win_rate': 0.9636363636363636, 'max_attempts': 6.890909090909091}}}, 6

Epochs:  95%|█████████▌| 19/20 [04:26<00:14, 14.18s/it]

Epoch 00019: reducing learning rate of group 0 to 1.5625e-06.
{4: {'easy': {'win': {'win_rate': 0.13333333333333333, 'max_attempts': 5.833333333333333}, 'lose': {'win_rate': 0.8666666666666667, 'max_attempts': 6.833333333333333}}, 'medium': {'win': {'win_rate': 0.13333333333333333, 'max_attempts': 5.833333333333333}, 'lose': {'win_rate': 0.8666666666666667, 'max_attempts': 6.833333333333333}}, 'hard': {'win': {'win_rate': 0.13333333333333333, 'max_attempts': 5.833333333333333}, 'lose': {'win_rate': 0.8666666666666667, 'max_attempts': 6.833333333333333}}}, 5: {'easy': {'win': {'win_rate': 0.03636363636363636, 'max_attempts': 5.890909090909091}, 'lose': {'win_rate': 0.9636363636363636, 'max_attempts': 6.890909090909091}}, 'medium': {'win': {'win_rate': 0.03636363636363636, 'max_attempts': 5.890909090909091}, 'lose': {'win_rate': 0.9636363636363636, 'max_attempts': 6.890909090909091}}, 'hard': {'win': {'win_rate': 0.03636363636363636, 'max_attempts': 5.890909090909091}, 'lose': {'win_rate

Epochs: 100%|██████████| 20/20 [04:40<00:00, 14.01s/it]

{4: {'easy': {'win': {'win_rate': 0.13333333333333333, 'max_attempts': 5.833333333333333}, 'lose': {'win_rate': 0.8666666666666667, 'max_attempts': 6.833333333333333}}, 'medium': {'win': {'win_rate': 0.13333333333333333, 'max_attempts': 5.833333333333333}, 'lose': {'win_rate': 0.8666666666666667, 'max_attempts': 6.833333333333333}}, 'hard': {'win': {'win_rate': 0.13333333333333333, 'max_attempts': 5.833333333333333}, 'lose': {'win_rate': 0.8666666666666667, 'max_attempts': 6.833333333333333}}}, 5: {'easy': {'win': {'win_rate': 0.03636363636363636, 'max_attempts': 5.890909090909091}, 'lose': {'win_rate': 0.9636363636363636, 'max_attempts': 6.890909090909091}}, 'medium': {'win': {'win_rate': 0.03636363636363636, 'max_attempts': 5.890909090909091}, 'lose': {'win_rate': 0.9636363636363636, 'max_attempts': 6.890909090909091}}, 'hard': {'win': {'win_rate': 0.03636363636363636, 'max_attempts': 5.890909090909091}, 'lose': {'win_rate': 0.9636363636363636, 'max_attempts': 6.890909090909091}}}, 6

In [ ]:
STOP

NameError: name 'STOP' is not defined

##### Tuning

In [ ]:
import torch
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
from scr.model import RNN
from torch.optim.lr_scheduler import StepLR
from scr.utils import *
from scr.early_stopping import EarlyStopping
import optuna

N_GAMES_PER_EPOCH = 10 ** 3
MAX_ATTEMPTS = 6
NUM_EPOCHS = 20

def objective(trial, dataset, static_config, num_epochs):
    dynamic_config = optuna_dynamic_hyperparameters(trial)
    config = {**static_config, **dynamic_config}
    # print(f"Trial {trial.number}: Configuration - {config}")

    kfold = KFold(n_splits=5, shuffle=True)
    aggregated_metrics = []

    best_objective_value = float('-inf')
    best_model_state = None

    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
        # print(f"Starting Fold {fold + 1}")
        train_loader, val_loader = get_data_loaders(dataset, train_idx, val_idx)
        model, optimizer = initialize_model(config)

        scheduler = StepLR(optimizer, step_size=trial.suggest_int("step_size", 5, 20), \
            gamma=trial.suggest_float("gamma", 0.1, 0.5))
        
        early_stopping = EarlyStopping(patience=10, delta=0.001)  # Early stopping instance


        for epoch in range(num_epochs):
            _, _ = train_one_epoch(model, train_loader)

            scheduler.step()
            win_rate, average_attempts, win_rate_by_length = simulate_games(
                    model=model,
                    word_list=None,  # Not using a word list in this scenario
                    char_to_idx=char_to_idx, 
                    idx_to_char=idx_to_char,
                    char_frequency=char_frequency, 
                    max_word_length=max_word_length, 
                    device=device, 
                    val_loader=val_loader,  # Pass the validation DataLoader
                    max_games_per_epoch=N_GAMES_PER_EPOCH,  # Number of games for validation
                    normalize=True, 
                    max_attempts=MAX_ATTEMPTS
            
            )

            for epoch in range(num_epochs):
    # Training step
    avg_actual_penalty, avg_miss_penalty = train_one_epoch(model, train_loader, device)

    print(f"Epoch {epoch+1}: Training data - Avg Actual Penalty: {avg_actual_penalty}, Avg Miss Penalty: {avg_miss_penalty}")

    # Validation step

    # win_rate, average_attempts, win_rate_by_length \
    #     = simulate_hangman_games(model, char_to_idx, idx_to_char, \
    #         char_frequency, max_word_length, device, word_list=None, \
    #     val_loader=test_loader, num_games=1000, max_attempts=6, normalize=True)

    
    avg_actual_penalty, avg_miss_penalty = train_one_epoch(model, train_loader, device)


    win_rate, average_attempts, win_rate_by_length \
        = simulate_hangman_games(model, char_to_idx, idx_to_char, \
            char_frequency, max_word_length, device, word_list=None, \
        val_loader=test_loader, num_games=1000, max_attempts=6, normalize=True)

    #         win_rate, average_attempts, win_rate_by_length = validate_one_epoch(model, \
    #             val_loader, char_to_idx, idx_to_char, char_frequency, max_word_length, device)

            aggregated_metrics.append((win_rate, average_attempts, win_rate_by_length))

            objective_value = calculate_objective_value(win_rate, \
                average_attempts, win_rate_by_length)
            
            trial.report(objective_value, epoch)

            if objective_value > best_objective_value:
                best_objective_value = objective_value
                best_model_state = model.state_dict()  # Update best model state

            # Check early stopping
            if early_stopping(objective_value):
                break  # Stop the current fold if early stopping criteria are met

            # Pruning check
            if trial.should_prune():
                return process_aggregated_metrics(aggregated_metrics)

    # Save the best model at the end of the trial
    if best_model_state:
        model.load_state_dict(best_model_state)
        model.save_model()  # Save the model using your custom method

    final_objective_value = process_aggregated_metrics(aggregated_metrics)
    
    return final_objective_value


def calculate_objective_value(win_rate_percentage, avg_attempts_per_game, \
    win_rate_by_word_length):
    # Define weights for each metric
    weight_for_win_rate = 0.7  # Higher weight for win rate
    weight_for_average_attempts = 0.3  # Lower weight for average attempts

    # Normalize win rate (converting percentage to a range of 0 to 1)
    normalized_win_rate = win_rate_percentage / 100

    # Normalize average attempts (assuming max attempts per game is known)
    max_attempts_per_game = 6  # Example: maximum number of attempts allowed
    normalized_avg_attempts = avg_attempts_per_game / max_attempts_per_game

    # Normalize win rate by word length
    normalized_win_rate_by_length = normalize_win_rate_by_length(win_rate_by_word_length)

    # Combine metrics into a single objective value
    objective_value = (weight_for_win_rate * normalized_win_rate) - \
                      (weight_for_average_attempts * normalized_avg_attempts) + \
                      normalized_win_rate_by_length
    return objective_value

def normalize_win_rate_by_length(win_rate_by_length_dict):
    # Example logic for normalization
    max_possible_win_rate = 1  # Maximum win rate (100% as a decimal)
    normalized_scores = {length: win_rate / max_possible_win_rate for \
        length, win_rate in win_rate_by_length_dict.items()}
    # Average the normalized win rates across all word lengths
    average_normalized_score = sum(normalized_scores.values()) / len(normalized_scores)
    return average_normalized_score

def process_aggregated_metrics(aggregated_metrics):
    # Initialize sums for each metric
    sum_win_rate = 0
    sum_average_attempts = 0
    sum_win_rate_by_length = {}

    # Process each set of metrics
    for win_rate, average_attempts, win_rate_by_length in aggregated_metrics:
        sum_win_rate += win_rate
        sum_average_attempts += average_attempts
        for length, rate in win_rate_by_length.items():
            sum_win_rate_by_length[length] = sum_win_rate_by_length.get(length, 0) + rate

    # Calculate averages
    num_entries = len(aggregated_metrics)
    avg_win_rate = sum_win_rate / num_entries
    avg_average_attempts = sum_average_attempts / num_entries
    avg_win_rate_by_length = {length: rate / num_entries for length, \
        rate in sum_win_rate_by_length.items()}

    # Now calculate the final objective value using these averages
    final_objective_value = calculate_objective_value(avg_win_rate, \
        avg_average_attempts, avg_win_rate_by_length)
    return final_objective_value


# Utility functions (for cleaner code)
def get_data_loaders(dataset, train_idx, val_idx):
    train_subset = Subset(dataset, train_idx)
    val_subset = Subset(dataset, val_idx)
    train_loader = DataLoader(train_subset, batch_size=32, \
        shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_subset, batch_size=32, \
        shuffle=False, collate_fn=collate_fn)
    return train_loader, val_loader

def initialize_model(config):
    model = RNN(config)
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
    model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    return model, optimizer

# Utility functions
def optuna_dynamic_hyperparameters(trial):
    # Define and return dynamic hyperparameters based on the trial
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    hidden_dim = trial.suggest_categorical('hidden_dim', [128, 256, 512])
    embedding_dim = trial.suggest_categorical('embedding_dim', [50, 100, 150])
    output_mid_features = trial.suggest_categorical('output_mid_features', [50, 100, 200])
    miss_linear_dim = trial.suggest_categorical('miss_linear_dim', [50, 100, 150])
    dropout = trial.suggest_float('dropout', 0.2, 0.5)
    num_layers = trial.suggest_int('num_layers', 1, 3)

    return {
        'lr': lr,
        'hidden_dim': hidden_dim,
        'embedding_dim': embedding_dim,
        'output_mid_features': output_mid_features,
        'miss_linear_dim': miss_linear_dim,
        'dropout': dropout,
        'num_layers': num_layers
    }

In [ ]:
from pathlib import Path

# Define your models directory path
models_dir = Path('models/notebook_2')

# Create the directory if it does not exist
models_dir.mkdir(parents=True, exist_ok=True)

# # Assuming missed_chars_tensor is a list of tensors for missed characters
# dataset = HangmanDataset(features, \
#     labels, missed_chars, original_words)

# Static configuration
static_config = {
    'rnn': 'LSTM',
    'vocab_size': 27,  # 26 English alphabets + 1 (e.g., for underscore)
    'use_embedding': True,  # Typically a design choice
    'input_feature_size': 5,  # Based on your feature engineering strategy
    'models': str(models_dir)
}

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define num_epochs for training in each fold
num_epochs = 4 # Adjust as needed
N_TRIAL = 3

# ====================================================== #
# Define the direction for the objective: 'maximize' or 'minimize'
direction = 'maximize'  # Adjust based on your objective function

study = optuna.create_study(direction=direction)
study.optimize(lambda trial: objective(trial, dataset, \
    static_config, num_epochs), n_trials=N_TRIAL)
# ====================================================== #

In [ ]:
best_params = study.best_params
# Output the best hyperparameters
print("Best trial:")
trial = study.best_trial
print(f"Value: {trial.value}")
print("Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

In [ ]:
best_trial = study.best_trial
print(f"Best trial number: {best_trial.number}")

# Assuming model saving includes the trial number in the filename
best_model_filename = f"models/trial_{best_trial.number+1}/LSTM.pth"
best_model = RNN.load_model(RNN, best_model_filename, device)

In [ ]:
STOP

In [ ]:
# Merge the two dictionaries to form a single configuration
config = {**static_config, **best_params}

config

In [ ]:
# Merge the two dictionaries to form a single configuration
config = {**static_config, **best_params}

# Initialize the RNN model with the combined configuration
model = RNN(config)

model = model.to(device)

# model.load_model(config['LSTM'], 2, 256, trial_number=10)

In [ ]:
STOP

##### Testing on unknown data

In [ ]:
%%capture

from scr.game import predict_next_character, simulate_game
from scr.model import RNN
import random
# random.seed(400)
# Your existing code for initializing the model, etc.

import random

def play_multiple_games(model, num_games, word_list, \
    char_to_idx, idx_to_char, char_frequency, max_word_length, device):
    game_results = []
    
    sampled_words = random.sample(word_list, num_games)  # Select unique words

    for random_word in sampled_words:
        with torch.no_grad():
            won, final_word, attempts_used = simulate_game(
                model, 
                random_word, 
                char_to_idx, 
                idx_to_char, 
                char_frequency, 
                max_word_length, 
                device, 
                normalize=True, 
                max_attempts=6
            )
        game_results.append((won, final_word, attempts_used))

    return game_results

# Example usage
num_games = 10**3
results = play_multiple_games(model, num_games, \
    word_list, char_to_idx, idx_to_char, \
        char_frequency, max_word_length, device)

# Analyzing results
total_wins = sum(result[0] for result in results)
win_rate = (total_wins / num_games) * 100
